# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
dataset_name = "Pseudomonas_analysis"
analysis_name = 'analysis_1'
NN_architecture = 'NN_2500_30'
file_prefix = 'Partition_corrected'
num_simulated_experiments = 600
lst_num_partitions = [1, 2, 3, 5, 10, 20,
                    30, 50, 70, 100, 200, 300, 400, 500, 600]
corrected = True
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input files
base_dir = os.path.abspath(
  os.path.join(
      os.getcwd(), "../.."))    # base dir on repo

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "input",
    "train_set_normalized.pcl")

experiment_ids_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    "experiment_ids.txt")

In [4]:
# Output files
similarity_corrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_1_similarity_corrected.pickle")

ci_corrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_1_ci_corrected.pickle")

In [5]:
# Run multiple simulations - corrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.matched_simulation_experiment_corrected)(i,
                                                           NN_architecture,
                                                           dataset_name,
                                                           analysis_name,
                                                           num_simulated_experiments,
                                                           lst_num_partitions,
                                                           corrected,
                                                           use_pca,
                                                           num_PCs,
                                                           file_prefix,
                                                           normalized_data_file,
                                                           experiment_ids_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 140.3min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 147.6min remaining: 221.4min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 148.2min remaining: 98.8min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 153.3min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 153.3min finished


In [6]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of partitions,,,,,
1,0.999993,0.999993,0.999992,0.999993,0.999993
2,0.997859,0.998244,0.998998,0.999306,0.998580
3,0.996675,0.997391,0.998265,0.997937,0.997002
5,0.992267,0.994618,0.994066,0.995776,0.993270
10,0.986732,0.986416,0.989203,0.990114,0.992153
20,0.966491,0.977740,0.973883,0.978544,0.978156
30,0.963603,0.970735,0.961566,0.962049,0.960981
50,0.944515,0.940228,0.942617,0.945481,0.948233
70,0.917413,0.928954,0.924565,0.924247,0.929358


In [7]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of partitions,
1,0.999993
2,0.998598
3,0.997454
5,0.993999
10,0.988924
20,0.974962
30,0.963787
50,0.944215
70,0.924908


In [8]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of partitions,
1,1.341109e-07
2,1.826246e-04
3,2.064779e-04
5,4.205537e-04
10,7.586091e-04
20,1.609888e-03
30,1.266259e-03
50,9.524265e-04
70,1.523958e-03


In [9]:
# Get confidence interval for each row (number of experiments)
err = std_scores*1.96

In [10]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of partitions,,
1,0.999993,0.999993
2,0.998240,0.998955
3,0.997049,0.997859
5,0.993175,0.994824
10,0.987437,0.990411
20,0.971807,0.978118
30,0.961305,0.966268
50,0.942348,0.946082
70,0.921921,0.927895


In [11]:
mean_scores

,score
number of partitions,
1,0.999993
2,0.998598
3,0.997454
5,0.993999
10,0.988924
20,0.974962
30,0.963787
50,0.944215
70,0.924908


In [12]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_corrected_file)
ci.to_pickle(ci_corrected_file)